***GENERATED CODE FOR regressionmodll PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import round
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'id', 'transformation_label': 'String Indexer'}], 'feature': 'id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'AVpf24SVLJeJML43FAIX', 'max': 'AVpiUMISilAPnD_xC-hu', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'id'}, {'feature_label': 'id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'asins', 'transformation_label': 'String Indexer'}], 'feature': 'asins', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B004OVE61U', 'max': 'B06XVDYHQX', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'asins'}, {'feature_label': 'asins', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('asins')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'brand', 'transformation_label': 'String Indexer'}], 'feature': 'brand', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Dell', 'max': 'Yamaha', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'brand'}, {'feature_label': 'brand', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('brand')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'categories', 'transformation_label': 'String Indexer'}], 'feature': 'categories', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'Electronics,Computers,Computer Accessories,Keyboards, Mice & Joysticks,Keyboards,All Keyboards,Computers & Accessories,Computer Accessories & Peripherals,Keyboards, Mice & Accessories,Computers & Tablets,iPad & Tablet Accessories,Tablet Cases, Covers & Keyboard Folios,Computer Peripherals,Keyboards Accessories,Computers Features,Cases, Covers & Keyboard Folios,Name Brands,Microsoft,Microsoft Surface,Microsoft Surface Accessories,Microsoft Surface Cases & Covers,Microsoft Surface Tablets,Frys,Tablets,Cases',
                                                                                                                                                                                                                                                                      'max': 'Electronics,Computers,Computer Accessories,Keyboards, Mice & Joysticks,Keyboards,All Keyboards,Computers & Accessories,Computer Accessories & Peripherals,Keyboards, Mice & Accessories,Computers & Tablets,iPad & Tablet Accessories,Tablet Cases, Covers & Keyboard Folios,Computer Peripherals,Keyboards Accessories,Computers Features,Cases, Covers & Keyboard Folios,Name Brands,Microsoft,Microsoft Surface,Microsoft Surface Accessories,Microsoft Surface Cases & Covers,Microsoft Surface Tablets,Frys,Tablets,Cases', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'categories'}, {'feature_label': 'categories', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('categories')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'colors', 'transformation_label': 'String Indexer'}], 'feature': 'colors', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '250', 'mean': '', 'stddev': '', 'min': 'Black', 'max': 'Navy,Black,Gray,Blue', 'missing': '404', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'colors'}, {'feature_label': 'colors', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('colors')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'dateAdded', 'transformation_label': 'Extract Date'}], 'feature': 'dateAdded', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'dateAdded'}, {'feature_label': 'dateAdded', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('dateAdded')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'dateUpdated', 'transformation_label': 'Extract Date'}], 'feature': 'dateUpdated', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'dateUpdated'}, {'feature_label': 'dateUpdated', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('dateUpdated')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'dimension', 'transformation_label': 'String Indexer'}], 'feature': 'dimension', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '113', 'mean': '', 'stddev': '', 'min': '2.1 in x 6.5 in x 3.1 in', 'max': '9.1 in x 5.9 in x 2.8 in', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'dimension'}, {'feature_label': 'dimension', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('dimension')
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ean', 'threshold': 2147483647.0, 'transformation_label': 'Binarizer'}], 'feature': 'ean', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
            'count': '500', 'mean': '890000000000.0', 'stddev': '0.0', 'min': '8.9E11', 'max': '8.9E11', 'missing': '307'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'ean'}, {'feature_label': 'ean', 'threshold': 2147483647.0, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('ean')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'imageURLs', 'transformation_label': 'String Indexer'}], 'feature': 'imageURLs', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': 'https://i5.walmartimages.com/asr/38f28958-7855-494a-babe-61513016efee_1.511282744b61af499f28ef9803d5d9c2.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/67fb7866-22f8-4409-a6c0-6f7e32951876_1.f815a47dd3a209a0c0ff302c1330e259.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/4a1e3f43-7b53-471a-b6ca-e7f6a4d3926f_1.9f99d56f7747f6b2690849bed4411002.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/thumbs/images/g/41YAAOSwZ~lZr5YB/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/~XIAAOSw4MNZlxNM/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/-G8AAOSw81dZqqdG/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/dzYAAOSw44BYHMo~/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/YfgAAOSwPK1ZUrs0/s-l96.jpg,https://i.ebayimg.com/images/g/F58AAOSwmtJXVY6u/s-l300.jpg,http://i.ebayimg.com/images/g/F58AAOSwmtJXVY6u/s-l64.jpg,http://i.ebayimg.com/images/g/MF8AAOSwtJZXVY6u/s-l64.jpg,http://i.ebayimg.com/images/g/pAEAAOSwbYZXVY6u/s-l64.jpg,https://static.bhphoto.com/images/itemImgPlaceholder.jpg,https://static.bhphoto.com/images/images500x500/sony_lbtgpx555_hi_fi_shelf_system_1401293128000_1038663.jpg,http://i.ebayimg.com/thumbs/images/g/hvEAAOSwxvFZdgQ4/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/socAAOSwRqBZnWog/s-l96.jpg,https://pisces.bbystatic.com/image2/BestBuy_US/images/products/8038/8038037_sa.jpg,http://images.frys.com/art/product/box_shots/8063794.box.GIF,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/8038/8038037_sa.jpg,http://i5.walmartimages.com/dfw/dce07b8c-93ac/k2-_3fd18f2a-8410-4266-9aaa-a70fe7661f47.v1.jpg,http://ecx.images-amazon.com/images/I/41f%2BoDYwlCL._SX300_QL70_.jpg,http://ecx.images-amazon.com/images/I/41f%2BoDYwlCL._SS40_.jpg,http://ecx.images-amazon.com/images/I/41L8PwLDgEL._SS40_.jpg,http://ecx.images-amazon.com/images/I/41yi%2BWezUEL._SS40_.jpg,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/8038/8038037_sa.jpg;canvasHeight=35;canvasWidth=105,http://i5.walmartimages.com/dfw/dce07b8c-8467/k2-_0e99e365-da8c-46e9-83b6-5e70614fea29.v1.jpg',
                                                                                                                                                                                                                                                                    'max': 'https://images-na.ssl-images-amazon.com/images/I/71Wg%2B%2BUUG5L._SL1500_.jpg,https://i5.walmartimages.com/asr/74730dee-ca4f-43c4-bd06-b5d39be3f916_1.fb1bf8086560d823e2e28b31b8231137.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/thumbs/images/g/wQAAAOSwKz9ZoNR2/s-l96.jpg,https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5744/5744203_sa.jpg,http://i.ebayimg.com/images/g/~uUAAOSwfRdZKBj3/s-l64.jpg,https://images-na.ssl-images-amazon.com/images/I/81WkGrNoRgL._SL1500_.jpg,http://i.ebayimg.com/thumbs/images/g/HWMAAOSwa9NZv~wp/s-l96.jpg,https://images-na.ssl-images-amazon.com/images/I/712Br6vh2CL._SL1500_.jpg,https://images-na.ssl-images-amazon.com/images/I/71Kyr8EaHSL._SL1500_.jpg,http://i.ebayimg.com/images/g/KX8AAOSwZKBZKBj3/s-l64.jpg,https://i.ebayimg.com/images/g/Yy4AAOSwRvdZa5oE/s-l300.jpg,http://i.ebayimg.com/thumbs/images/g/UvMAAOSwknVZegQV/s-l96.jpg,https://images-na.ssl-images-amazon.com/images/I/71eyBPe5zoL._SL1500_.jpg,https://images-na.ssl-images-amazon.com/images/I/91tg2H3QDzL._SL1500_.jpg,http://i.ebayimg.com/thumbs/images/g/BQMAAOSwi~5Zx9y2/s-l96.jpg,https://i5.walmartimages.com/asr/59599337-f9ca-42b8-8cf7-dc892415fd09_1.37e5d0bacf76a9d394311f1ed2778f9b.png?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://static.bhphoto.com/images/images500x500/sony_srsxb30_blk_srsxb30_extra_bass_wireless_1490068317000_1319172.jpg,http://i.ebayimg.com/thumbs/images/g/PGEAAOSwyXhZvxyH/s-l96.jpg,https://static.bhphoto.com/images/itemImgPlaceholder.jpg,http://i.ebayimg.com/thumbs/images/g/T5kAAOSwjPdZv~u7/s-l96.jpg,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/5744/5744203_sa.jpg,https://i5.walmartimages.com/asr/5d70386d-1caa-4368-b3a3-bdf512b2b4cb_1.42843e7562bcf53ecbdade9b05a44d1a.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/thumbs/images/g/hvEAAOSwxvFZdgQ4/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/-SAAAOSwOudZu07R/s-l96.jpg,https://i5.walmartimages.com/asr/7996f6e4-a928-4355-83ae-54909d2bfde6_1.42843e7562bcf53ecbdade9b05a44d1a.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/thumbs/images/g/Rb0AAOSwSTFZbP7w/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/v1cAAOSwJmVZvYLm/s-l96.jpg,https://i.ebayimg.com/images/g/stQAAOSwR29ZKBj4/s-l300.jpg,https://images-na.ssl-images-amazon.com/images/I/81ww8gkRaVL._SL1500_.jpg,https://images-na.ssl-images-amazon.com/images/I/71XE3bK8xYL._SL1500_.jpg,http://i.ebayimg.com/images/g/RH4AAOSw3gJZKBj4/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/o2oAAOSwvqlZefB3/s-l96.jpg,http://i.ebayimg.com/images/g/stQAAOSwR29ZKBj4/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/SKkAAOSw3ONZoxW7/s-l96.jpg,https://images-na.ssl-images-amazon.com/images/I/91OKUt-5quL._SL1500_.jpg,http://i.ebayimg.com/thumbs/images/g/wJUAAOSw1P9Zb-20/s-l96.jpg,http://i.ebayimg.com/images/g/Yy4AAOSwRvdZa5oE/s-l64.jpg,https://i5.walmartimages.com/asr/7dc23d04-e7d6-4174-a78f-696525fd56c0_1.fb1bf8086560d823e2e28b31b8231137.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/thumbs/images/g/CH4AAOSwfa9ZqHlv/s-l96.jpg,https://images-na.ssl-images-amazon.com/images/I/91SeWQuYahL._SL1500_.jpg,https://i5.walmartimages.com/asr/d90cf8e4-655e-4a69-8bb2-ca449de20f1e_1.42843e7562bcf53ecbdade9b05a44d1a.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://images-na.ssl-images-amazon.com/images/I/71oFX6M-UxL._SL1500_.jpg,http://i.ebayimg.com/thumbs/images/g/adkAAOSw0xRZoDqh/s-l96.jpg,http://i.ebayimg.com/images/g/yccAAOSwpINZa5oU/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/8HMAAOSwfRdZODHr/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/~tEAAOSwguxZeR3b/s-l96.jpg,https://i5.walmartimages.com/asr/4f436400-e3fb-4a10-813a-975134120845_1.d069b0d307e149d56c48fbbe97643d13.png?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/images/g/OygAAOSwn-tZKBj6/s-l64.jpg,https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5744/5744203_sd.jpg,http://i.ebayimg.com/images/g/2VkAAOSws5pZKBj8/s-l64.jpg,https://images-na.ssl-images-amazon.com/images/I/71nzyyDE2iL._SL1500_.jpg,https://images-na.ssl-images-amazon.com/images/I/81RBMyoUKpL._SL1500_.jpg,http://i.ebayimg.com/images/g/Cd0AAOSwcgNZKBj3/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/41YAAOSwZ~lZr5YB/s-l96.jpg,http://i.ebayimg.com/images/g/4zcAAOSwYvFZKBj4/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/a-gAAOSwrNlZvYNR/s-l96.jpg,https://images-na.ssl-images-amazon.com/images/I/810XztMZV3L._SL1500_.jpg,https://images-na.ssl-images-amazon.com/images/I/81VR%2Bbpi8aL._SL1500_.jpg,http://i.ebayimg.com/thumbs/images/g/5t8AAOSw62VZw0g4/s-l96.jpg,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/5744/5744203_sd.jpg,http://i.ebayimg.com/images/g/3q0AAOSwsXFZKBj4/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/P8oAAOSwsXVZa5ae/s-l96.jpg,http://i.ebayimg.com/images/g/n6cAAOSwrDBZa5oN/s-l64.jpg,https://i5.walmartimages.com/asr/8cffe396-95ca-4e9f-aeb6-bbb495b7e7ab_1.fb1bf8086560d823e2e28b31b8231137.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'imageURLs'}, {'feature_label': 'imageURLs', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('imageURLs')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'keys', 'transformation_label': 'String Indexer'}], 'feature': 'keys', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': '047532907711,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/4525009,ihome/ibt33bc,sditechnologies/ibt33bc,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/555013051,ihomerechargeablesplashproofstereobluetoothspeakerblackibt33bc/b00wbs31oq,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/b00wbs31oq',
                                                                                                                                                                                                                                                          'max': 'verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/b0172ds3me,novatel/mifi6620l,verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/9940096,649496019587,verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/b00ntjkaxg,0649496019587,verizonwireless/mifi6620l', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'keys'}, {'feature_label': 'keys', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('keys')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'manufacturer', 'transformation_label': 'String Indexer'}], 'feature': 'manufacturer', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '246', 'mean': '', 'stddev': '', 'min': 'Glengery', 'max': 'Yamaha', 'missing': '239', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'manufacturer'}, {'feature_label': 'manufacturer', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('manufacturer')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'manufacturerNumber', 'transformation_label': 'String Indexer'}], 'feature': 'manufacturerNumber', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '45W SLIM AC ADAPTER - 88801419', 'max': 'Mifi6620L', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'manufacturerNumber'}, {'feature_label': 'manufacturerNumber', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('manufacturerNumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'name', 'transformation_label': 'String Indexer'}], 'feature': 'name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Microsoft Surface Pro 4 Type Cover with Fingerprint ID', 'max': 'Microsoft Surface Pro 4 Type Cover with Fingerprint ID', 'missing': '0', 'distinct': '11'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'name'}, {'feature_label': 'name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'primaryCategories', 'transformation_label': 'String Indexer'}], 'feature': 'primaryCategories', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Electronics', 'max': 'Electronics', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'primaryCategories'}, {'feature_label': 'primaryCategories', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('primaryCategories')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewsdate', 'transformation_label': 'Extract Date'}], 'feature': 'reviewsdate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'reviewsdate'}, {'feature_label': 'reviewsdate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('reviewsdate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewsdateSeen', 'transformation_label': 'String Indexer'}], 'feature': 'reviewsdateSeen', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '2015-07-10T00:00:00Z', 'max': '2018-04-27T00:00:00Z', 'missing': '0', 'distinct': '55'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reviewsdateSeen'}, {'feature_label': 'reviewsdateSeen', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reviewsdateSeen')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewssourceURLs', 'transformation_label': 'String Indexer'}], 'feature': 'reviewssourceURLs', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'http://reviews.bestbuy.com/3545/2877396/reviews.htm?format=embedded,https://reviews.bestbuy.com/3545/2877396/reviews.htm?format=embedded', 'max': 'https://www.walmart.com/reviews/product/51933274', 'missing': '0', 'distinct': '201'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reviewssourceURLs'}, {'feature_label': 'reviewssourceURLs', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reviewssourceURLs')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewstext', 'transformation_label': 'String Indexer'}], 'feature': 'reviewstext', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '499', 'mean': '', 'stddev': '', 'min': '"...power supply is a little low for an XPS with a 7th gen i7. All the specs are great, and video card is more that adequate, but if the XPS series is ""built"" to allow for upgrades',
                                                                                                                                                                                                                                                                        'max': 'we bought the camcorder from Best Buy years ago and they help us get the tapes on line', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reviewstext'}, {'feature_label': 'reviewstext', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reviewstext')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewstitle', 'transformation_label': 'String Indexer'}], 'feature': 'reviewstitle', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': ' CarPlay)', 'max': 'wireless', 'missing': '0', 'distinct': '465'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reviewstitle'}, {'feature_label': 'reviewstitle', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reviewstitle')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'reviewsusername', 'transformation_label': 'String Indexer'}], 'feature': 'reviewsusername', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '5124.0', 'stddev': 'nan', 'min': ' but I really like the lights on this speaker. There is the outer ring that surrounds the speaker on the front that changes different colors and two white LED ""strobe"" lights on either side. You can set the lights to dance with the music', 'max': 'zombiepie69-7', 'missing': '0', 'distinct': '489'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'reviewsusername'}, {'feature_label': 'reviewsusername', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('reviewsusername')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'sourceURLs', 'transformation_label': 'String Indexer'}], 'feature': 'sourceURLs', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '', 'stddev': '', 'min': ' set a more calming lighting effect or turn off the lights completely by using the SongPal app. The lights really to add a bit of a party atmosphere and helps this speaker stand out from other brands.+ Bass booster: A quick press of ""Extra Bass"" button substantially increases lows and adds a great bass effect', 'max': 'http://www.walmart.com/ip/Sony-LBT-GPX555-Mini-System-with-Bluetooth-and-NFC/40588069,https://www.walmart.com/ip/Sony-LBT-GPX555-Mini-System-with-Bluetooth-and-NFC/40588069,http://www.ebay.com/itm/Sony-LBT-GPX555-1800W-Bluetooth-Wireless-Music-System/331874713008,https://www.walmart.com/reviews/product/40588069,https://www.bhphotovideo.com/c/product/1038663-REG/sony_lbtgpx555_hi_fi_shelf_system.html,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?skuId=8038037,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=4,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=2,http://www.frys.com/product/8063794?site=sr:SEARCH:MAIN_RSLT_PG,https://www.amazon.com/dp/B00IWQ3YPC,http://www.amazon.com/Sony-LBT-GPX555-Power-System-Bluetooth/dp/B00IWQ3YPC,https://www.amazon.com/product-reviews/B00IWQ3YPC/,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=10,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=9,http://www.frys.com/product/8063794,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=11,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=8,http://frys.com/product/8063794#detailed,http://www.walmart.com/ip/Sony-Lbt-gpx555-Mini-Hi-fi-System-1800-W-Rms-Black-Cd-Player-30-Channel-s-Am-Fm-Video-Cd-Sacd-Dvd-Video-2.1-Speaker-s-Cd-da-Mp3/46145643,http://www.newegg.com/Product/Product.aspx?Item=9SIA0AJ3MD4812,http://www.newegg.com/Product/Product.aspx?Item=N82E16882105737,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=5,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037#tab=buyingOptions?bof=openbox,http://www.bestbuy.com:80/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037#tabbed-customerreviews,http://www.bestbuy.com:80/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037#tabbed-customerreviews,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=3,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037&reviewsPage=2,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=6,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037&reviewsPage=4,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037&reviewsPage=3,http://www.walmart.com/ip/Sony-LBT-GPX555-Mini-System-with-Bluetooth-and-NFC/40588069?action=product_interest&action_type=title&item_id=40588069&placement_id=irs-2-b1&strategy=TIC&visitor_id&category=&customer_id_enc&config_id=2&parent_item_id=20370545&parent_anchor_item_id=20370545&bucket_id=irsbucketdefault&beacon_version=1.0.1&findingMethod=p13n,http://www.walmart.com/ip/Sony-LBT-GPX555-Mini-System-with-Bluetooth-and-NFC/40588069?action=product_interest&action_type=title&item_id=40588069&placement_id=irs-2-b1&strategy=TIC&visitor_id&category=&customer_id_enc&config_id=2&parent_item_id=20370545&parent_anchor_item_id=20370545&bucket_id=irsbucketdefault&beacon_version=1.0.1&findingMethod=p13n#reviews,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=7,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=12,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=13', 'missing': '0', 'distinct': '19'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'sourceURLs'}, {'feature_label': 'sourceURLs', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('sourceURLs')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'upc', 'transformation_label': 'String Indexer'}], 'feature': 'upc', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '890000000000.0', 'stddev': '0.0', 'min': ' it works most of the time and is a quicker way to ""sign-on"" than entering my 20 character password!The keyboard easily comes off and on with the tablet and it is magnetized allowing for the keyboard to be at an angle compatible with typing while I was in-flight."', 'max': 'wNg11', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'upc'}, {'feature_label': 'upc', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('upc')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'weight', 'transformation_label': 'String Indexer'}], 'feature': 'weight', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '500', 'mean': '27242902695.0', 'stddev': '0.0', 'min': ' it can make some songs sound a little ""muddy"" with an overemphasis on lows.+ NFC pairing: If your phone features an NFC chip', 'max': 'http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=13,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=14,https://www.bestbuy.com/site/sony-xb30-portable-bluetooth-speaker-black/5744203.p?skuId=5744203,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=15,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=16,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=17,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=18,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=19,http://www.bestbuy.com/site/products/5744203.p,http://www.ebay.com/itm/Sony-SRS-XB30-Portable-Bluetooth-Wireless-Speaker-Black/272855583204,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=20,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=21,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=22,https://www.amazon.com/Sony-Portable-Wireless-Speaker-Bluetooth/dp/B01MUG578I/,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=23,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=46,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=47,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=48,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=49,https://www.bestbuy.com/site/products/5744203.p,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=50,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=51,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=52,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=53,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=54,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=10,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=55,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=11,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=56,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=12,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=35,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=36,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=37,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=38,https://www.amazon.com/Sony-SRSXB30-BLK-Portable-Bluetooth/dp/B01MUG578I/,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=39,https://www.amazon.com/Sony-Bluetooth-SRS-XB30-BLK-Refurbished/dp/B072MP2CFG/,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=4,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=5,https://www.bhphotovideo.com/c/product/1319172-REG/sony_srsxb30_blk_srsxb30_extra_bass_wireless.html,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=2,https://www.walmart.com/ip/SRSXB30-BLK/697156213,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=3,http://www.bestbuy.com/site/sony-xb30-portable-bluetooth-speaker-black/5744203.p?skuId=5744203,http://www.ebay.com/itm/Sony-XB30-Portable-Wireless-Speaker-with-Bluetooth-Black-2017-model/311881660787?hash=item489d987973%3Ag%3AstQAAOSwR29ZKBj4,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=40,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=41,https://www.walmart.com/ip/SONY-SRS-XB30-BLK-Portable-Wireless-Speaker/697156213,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=42,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=8,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=43,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=9,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=44,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=6,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=45,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=7,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=24,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=25,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=26,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=27,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=28,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=29,http://www.ebay.com/itm/Sony-SRS-XB30-Portable-Bluetooth-Wireless-Speaker-Black-SRSXB30/282573811327,http://www.ebay.com/itm/Sony-SRS-XB30-Portable-Bluetooth-Wireless-Speaker-Black-SRSXB30/282573811327?hash=item41cab62e7f%3Ag%3AYy4AAOSwRvdZa5oE,http://www.walmart.com/ip/SONY-SRS-XB30-BLK-Portable-Wireless-Speaker/697156213,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=30,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=31,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=32,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=33,https://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=34', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'weight'}, {'feature_label': 'weight', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('weight')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run regressionmodllHooks.ipynb
try:
	#sourcePreExecutionHook()

	electronicsproductdatacleaned = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/ElectronicsProductData_Cleaned.csv', 'filename': 'ElectronicsProductData_Cleaned.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(electronicsproductdatacleaned)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run regressionmodllHooks.ipynb
try:
	#transformationPreExecutionHook()

	regressionmodllautofe = TransformationMain.run(electronicsproductdatacleaned,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "id", "transformation_label": "String Indexer"}], "feature": "id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AVpf24SVLJeJML43FAIX", "max": "AVpiUMISilAPnD_xC-hu", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "id"}, {"transformationsData": [{"feature_label": "asins", "transformation_label": "String Indexer"}], "feature": "asins", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B004OVE61U", "max": "B06XVDYHQX", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "asins"}, {"transformationsData": [{"feature_label": "brand", "transformation_label": "String Indexer"}], "feature": "brand", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Dell", "max": "Yamaha", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "brand"}, {"transformationsData": [{"feature_label": "categories", "transformation_label": "String Indexer"}], "feature": "categories", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Electronics,Computers,Computer Accessories,Keyboards, Mice & Joysticks,Keyboards,All Keyboards,Computers & Accessories,Computer Accessories & Peripherals,Keyboards, Mice & Accessories,Computers & Tablets,iPad & Tablet Accessories,Tablet Cases, Covers & Keyboard Folios,Computer Peripherals,Keyboards Accessories,Computers Features,Cases, Covers & Keyboard Folios,Name Brands,Microsoft,Microsoft Surface,Microsoft Surface Accessories,Microsoft Surface Cases & Covers,Microsoft Surface Tablets,Frys,Tablets,Cases", "max": "Electronics,Computers,Computer Accessories,Keyboards, Mice & Joysticks,Keyboards,All Keyboards,Computers & Accessories,Computer Accessories & Peripherals,Keyboards, Mice & Accessories,Computers & Tablets,iPad & Tablet Accessories,Tablet Cases, Covers & Keyboard Folios,Computer Peripherals,Keyboards Accessories,Computers Features,Cases, Covers & Keyboard Folios,Name Brands,Microsoft,Microsoft Surface,Microsoft Surface Accessories,Microsoft Surface Cases & Covers,Microsoft Surface Tablets,Frys,Tablets,Cases", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "categories"}, {"transformationsData": [{"feature_label": "colors", "transformation_label": "String Indexer"}], "feature": "colors", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "250", "mean": "", "stddev": "", "min": "Black", "max": "Navy,Black,Gray,Blue", "missing": "404", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "colors"}, {"transformationsData": [{"feature_label": "dateAdded", "transformation_label": "Extract Date"}], "feature": "dateAdded", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "dateAdded"}, {"transformationsData": [{"feature_label": "dateUpdated", "transformation_label": "Extract Date"}], "feature": "dateUpdated", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "dateUpdated"}, {"transformationsData": [{"feature_label": "dimension", "transformation_label": "String Indexer"}], "feature": "dimension", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "113", "mean": "", "stddev": "", "min": "2.1 in x 6.5 in x 3.1 in", "max": "9.1 in x 5.9 in x 2.8 in", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "dimension"}, {"transformationsData": [{"feature_label": "ean", "threshold": 2147483647.0, "transformation_label": "Binarizer"}], "feature": "ean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "890000000000.0", "stddev": "0.0", "min": "8.9E11", "max": "8.9E11", "missing": "307"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "ean"}, {"transformationsData": [{"feature_label": "imageURLs", "transformation_label": "String Indexer"}], "feature": "imageURLs", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "https://i5.walmartimages.com/asr/38f28958-7855-494a-babe-61513016efee_1.511282744b61af499f28ef9803d5d9c2.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/67fb7866-22f8-4409-a6c0-6f7e32951876_1.f815a47dd3a209a0c0ff302c1330e259.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://i5.walmartimages.com/asr/4a1e3f43-7b53-471a-b6ca-e7f6a4d3926f_1.9f99d56f7747f6b2690849bed4411002.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/thumbs/images/g/41YAAOSwZ~lZr5YB/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/~XIAAOSw4MNZlxNM/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/-G8AAOSw81dZqqdG/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/dzYAAOSw44BYHMo~/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/YfgAAOSwPK1ZUrs0/s-l96.jpg,https://i.ebayimg.com/images/g/F58AAOSwmtJXVY6u/s-l300.jpg,http://i.ebayimg.com/images/g/F58AAOSwmtJXVY6u/s-l64.jpg,http://i.ebayimg.com/images/g/MF8AAOSwtJZXVY6u/s-l64.jpg,http://i.ebayimg.com/images/g/pAEAAOSwbYZXVY6u/s-l64.jpg,https://static.bhphoto.com/images/itemImgPlaceholder.jpg,https://static.bhphoto.com/images/images500x500/sony_lbtgpx555_hi_fi_shelf_system_1401293128000_1038663.jpg,http://i.ebayimg.com/thumbs/images/g/hvEAAOSwxvFZdgQ4/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/socAAOSwRqBZnWog/s-l96.jpg,https://pisces.bbystatic.com/image2/BestBuy_US/images/products/8038/8038037_sa.jpg,http://images.frys.com/art/product/box_shots/8063794.box.GIF,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/8038/8038037_sa.jpg,http://i5.walmartimages.com/dfw/dce07b8c-93ac/k2-_3fd18f2a-8410-4266-9aaa-a70fe7661f47.v1.jpg,http://ecx.images-amazon.com/images/I/41f%2BoDYwlCL._SX300_QL70_.jpg,http://ecx.images-amazon.com/images/I/41f%2BoDYwlCL._SS40_.jpg,http://ecx.images-amazon.com/images/I/41L8PwLDgEL._SS40_.jpg,http://ecx.images-amazon.com/images/I/41yi%2BWezUEL._SS40_.jpg,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/8038/8038037_sa.jpg;canvasHeight=35;canvasWidth=105,http://i5.walmartimages.com/dfw/dce07b8c-8467/k2-_0e99e365-da8c-46e9-83b6-5e70614fea29.v1.jpg", "max": "https://images-na.ssl-images-amazon.com/images/I/71Wg%2B%2BUUG5L._SL1500_.jpg,https://i5.walmartimages.com/asr/74730dee-ca4f-43c4-bd06-b5d39be3f916_1.fb1bf8086560d823e2e28b31b8231137.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/thumbs/images/g/wQAAAOSwKz9ZoNR2/s-l96.jpg,https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5744/5744203_sa.jpg,http://i.ebayimg.com/images/g/~uUAAOSwfRdZKBj3/s-l64.jpg,https://images-na.ssl-images-amazon.com/images/I/81WkGrNoRgL._SL1500_.jpg,http://i.ebayimg.com/thumbs/images/g/HWMAAOSwa9NZv~wp/s-l96.jpg,https://images-na.ssl-images-amazon.com/images/I/712Br6vh2CL._SL1500_.jpg,https://images-na.ssl-images-amazon.com/images/I/71Kyr8EaHSL._SL1500_.jpg,http://i.ebayimg.com/images/g/KX8AAOSwZKBZKBj3/s-l64.jpg,https://i.ebayimg.com/images/g/Yy4AAOSwRvdZa5oE/s-l300.jpg,http://i.ebayimg.com/thumbs/images/g/UvMAAOSwknVZegQV/s-l96.jpg,https://images-na.ssl-images-amazon.com/images/I/71eyBPe5zoL._SL1500_.jpg,https://images-na.ssl-images-amazon.com/images/I/91tg2H3QDzL._SL1500_.jpg,http://i.ebayimg.com/thumbs/images/g/BQMAAOSwi~5Zx9y2/s-l96.jpg,https://i5.walmartimages.com/asr/59599337-f9ca-42b8-8cf7-dc892415fd09_1.37e5d0bacf76a9d394311f1ed2778f9b.png?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://static.bhphoto.com/images/images500x500/sony_srsxb30_blk_srsxb30_extra_bass_wireless_1490068317000_1319172.jpg,http://i.ebayimg.com/thumbs/images/g/PGEAAOSwyXhZvxyH/s-l96.jpg,https://static.bhphoto.com/images/itemImgPlaceholder.jpg,http://i.ebayimg.com/thumbs/images/g/T5kAAOSwjPdZv~u7/s-l96.jpg,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/5744/5744203_sa.jpg,https://i5.walmartimages.com/asr/5d70386d-1caa-4368-b3a3-bdf512b2b4cb_1.42843e7562bcf53ecbdade9b05a44d1a.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/thumbs/images/g/hvEAAOSwxvFZdgQ4/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/-SAAAOSwOudZu07R/s-l96.jpg,https://i5.walmartimages.com/asr/7996f6e4-a928-4355-83ae-54909d2bfde6_1.42843e7562bcf53ecbdade9b05a44d1a.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/thumbs/images/g/Rb0AAOSwSTFZbP7w/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/v1cAAOSwJmVZvYLm/s-l96.jpg,https://i.ebayimg.com/images/g/stQAAOSwR29ZKBj4/s-l300.jpg,https://images-na.ssl-images-amazon.com/images/I/81ww8gkRaVL._SL1500_.jpg,https://images-na.ssl-images-amazon.com/images/I/71XE3bK8xYL._SL1500_.jpg,http://i.ebayimg.com/images/g/RH4AAOSw3gJZKBj4/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/o2oAAOSwvqlZefB3/s-l96.jpg,http://i.ebayimg.com/images/g/stQAAOSwR29ZKBj4/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/SKkAAOSw3ONZoxW7/s-l96.jpg,https://images-na.ssl-images-amazon.com/images/I/91OKUt-5quL._SL1500_.jpg,http://i.ebayimg.com/thumbs/images/g/wJUAAOSw1P9Zb-20/s-l96.jpg,http://i.ebayimg.com/images/g/Yy4AAOSwRvdZa5oE/s-l64.jpg,https://i5.walmartimages.com/asr/7dc23d04-e7d6-4174-a78f-696525fd56c0_1.fb1bf8086560d823e2e28b31b8231137.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/thumbs/images/g/CH4AAOSwfa9ZqHlv/s-l96.jpg,https://images-na.ssl-images-amazon.com/images/I/91SeWQuYahL._SL1500_.jpg,https://i5.walmartimages.com/asr/d90cf8e4-655e-4a69-8bb2-ca449de20f1e_1.42843e7562bcf53ecbdade9b05a44d1a.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF,https://images-na.ssl-images-amazon.com/images/I/71oFX6M-UxL._SL1500_.jpg,http://i.ebayimg.com/thumbs/images/g/adkAAOSw0xRZoDqh/s-l96.jpg,http://i.ebayimg.com/images/g/yccAAOSwpINZa5oU/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/8HMAAOSwfRdZODHr/s-l96.jpg,http://i.ebayimg.com/thumbs/images/g/~tEAAOSwguxZeR3b/s-l96.jpg,https://i5.walmartimages.com/asr/4f436400-e3fb-4a10-813a-975134120845_1.d069b0d307e149d56c48fbbe97643d13.png?odnHeight=450&odnWidth=450&odnBg=FFFFFF,http://i.ebayimg.com/images/g/OygAAOSwn-tZKBj6/s-l64.jpg,https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5744/5744203_sd.jpg,http://i.ebayimg.com/images/g/2VkAAOSws5pZKBj8/s-l64.jpg,https://images-na.ssl-images-amazon.com/images/I/71nzyyDE2iL._SL1500_.jpg,https://images-na.ssl-images-amazon.com/images/I/81RBMyoUKpL._SL1500_.jpg,http://i.ebayimg.com/images/g/Cd0AAOSwcgNZKBj3/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/41YAAOSwZ~lZr5YB/s-l96.jpg,http://i.ebayimg.com/images/g/4zcAAOSwYvFZKBj4/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/a-gAAOSwrNlZvYNR/s-l96.jpg,https://images-na.ssl-images-amazon.com/images/I/810XztMZV3L._SL1500_.jpg,https://images-na.ssl-images-amazon.com/images/I/81VR%2Bbpi8aL._SL1500_.jpg,http://i.ebayimg.com/thumbs/images/g/5t8AAOSw62VZw0g4/s-l96.jpg,http://pisces.bbystatic.com/image2/BestBuy_US/images/products/5744/5744203_sd.jpg,http://i.ebayimg.com/images/g/3q0AAOSwsXFZKBj4/s-l64.jpg,http://i.ebayimg.com/thumbs/images/g/P8oAAOSwsXVZa5ae/s-l96.jpg,http://i.ebayimg.com/images/g/n6cAAOSwrDBZa5oN/s-l64.jpg,https://i5.walmartimages.com/asr/8cffe396-95ca-4e9f-aeb6-bbb495b7e7ab_1.fb1bf8086560d823e2e28b31b8231137.jpeg?odnHeight=450&odnWidth=450&odnBg=FFFFFF", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "imageURLs"}, {"transformationsData": [{"feature_label": "keys", "transformation_label": "String Indexer"}], "feature": "keys", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "047532907711,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/4525009,ihome/ibt33bc,sditechnologies/ibt33bc,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/555013051,ihomerechargeablesplashproofstereobluetoothspeakerblackibt33bc/b00wbs31oq,ihomerechargeablesplashproofstereobluetoothspeakerwithspeakerphone/b00wbs31oq", "max": "verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/b0172ds3me,novatel/mifi6620l,verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/9940096,649496019587,verizonmifi6620ljetpack4gltemobilehotspotverizonwireless/b00ntjkaxg,0649496019587,verizonwireless/mifi6620l", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "keys"}, {"transformationsData": [{"feature_label": "manufacturer", "transformation_label": "String Indexer"}], "feature": "manufacturer", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "246", "mean": "", "stddev": "", "min": "Glengery", "max": "Yamaha", "missing": "239", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "manufacturer"}, {"transformationsData": [{"feature_label": "manufacturerNumber", "transformation_label": "String Indexer"}], "feature": "manufacturerNumber", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "45W SLIM AC ADAPTER - 88801419", "max": "Mifi6620L", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "manufacturerNumber"}, {"transformationsData": [{"feature_label": "name", "transformation_label": "String Indexer"}], "feature": "name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Microsoft Surface Pro 4 Type Cover with Fingerprint ID", "max": "Microsoft Surface Pro 4 Type Cover with Fingerprint ID", "missing": "0", "distinct": "11"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "name"}, {"transformationsData": [{"feature_label": "primaryCategories", "transformation_label": "String Indexer"}], "feature": "primaryCategories", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Electronics", "max": "Electronics", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "primaryCategories"}, {"transformationsData": [{"feature_label": "reviewsdate", "transformation_label": "Extract Date"}], "feature": "reviewsdate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "reviewsdate"}, {"transformationsData": [{"feature_label": "reviewsdateSeen", "transformation_label": "String Indexer"}], "feature": "reviewsdateSeen", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "2015-07-10T00:00:00Z", "max": "2018-04-27T00:00:00Z", "missing": "0", "distinct": "55"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reviewsdateSeen"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsdoRecommend", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "492", "mean": "0.94", "stddev": "0.23", "min": "0", "max": "1", "missing": "59"}, "updatedLabel": "reviewsdoRecommend"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsnumHelpful", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "426", "mean": "0.11", "stddev": "0.78", "min": "0", "max": "12", "missing": "74"}, "updatedLabel": "reviewsnumHelpful"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "reviewsrating", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "492", "mean": "4.59", "stddev": "0.8", "min": "1", "max": "5", "missing": "8"}, "updatedLabel": "reviewsrating"}, {"transformationsData": [{"feature_label": "reviewssourceURLs", "transformation_label": "String Indexer"}], "feature": "reviewssourceURLs", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "http://reviews.bestbuy.com/3545/2877396/reviews.htm?format=embedded,https://reviews.bestbuy.com/3545/2877396/reviews.htm?format=embedded", "max": "https://www.walmart.com/reviews/product/51933274", "missing": "0", "distinct": "201"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reviewssourceURLs"}, {"transformationsData": [{"feature_label": "reviewstext", "transformation_label": "String Indexer"}], "feature": "reviewstext", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "499", "mean": "", "stddev": "", "min": "\"...power supply is a little low for an XPS with a 7th gen i7. All the specs are great, and video card is more that adequate, but if the XPS series is \"\"built\"\" to allow for upgrades", "max": "we bought the camcorder from Best Buy years ago and they help us get the tapes on line", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reviewstext"}, {"transformationsData": [{"feature_label": "reviewstitle", "transformation_label": "String Indexer"}], "feature": "reviewstitle", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " CarPlay)", "max": "wireless", "missing": "0", "distinct": "465"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reviewstitle"}, {"transformationsData": [{"feature_label": "reviewsusername", "transformation_label": "String Indexer"}], "feature": "reviewsusername", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "5124.0", "stddev": "nan", "min": " but I really like the lights on this speaker. There is the outer ring that surrounds the speaker on the front that changes different colors and two white LED \"\"strobe\"\" lights on either side. You can set the lights to dance with the music", "max": "zombiepie69-7", "missing": "0", "distinct": "489"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reviewsusername"}, {"transformationsData": [{"feature_label": "sourceURLs", "transformation_label": "String Indexer"}], "feature": "sourceURLs", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": " set a more calming lighting effect or turn off the lights completely by using the SongPal app. The lights really to add a bit of a party atmosphere and helps this speaker stand out from other brands.+ Bass booster: A quick press of \"\"Extra Bass\"\" button substantially increases lows and adds a great bass effect", "max": "http://www.walmart.com/ip/Sony-LBT-GPX555-Mini-System-with-Bluetooth-and-NFC/40588069,https://www.walmart.com/ip/Sony-LBT-GPX555-Mini-System-with-Bluetooth-and-NFC/40588069,http://www.ebay.com/itm/Sony-LBT-GPX555-1800W-Bluetooth-Wireless-Music-System/331874713008,https://www.walmart.com/reviews/product/40588069,https://www.bhphotovideo.com/c/product/1038663-REG/sony_lbtgpx555_hi_fi_shelf_system.html,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?skuId=8038037,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=4,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=2,http://www.frys.com/product/8063794?site=sr:SEARCH:MAIN_RSLT_PG,https://www.amazon.com/dp/B00IWQ3YPC,http://www.amazon.com/Sony-LBT-GPX555-Power-System-Bluetooth/dp/B00IWQ3YPC,https://www.amazon.com/product-reviews/B00IWQ3YPC/,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=10,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=9,http://www.frys.com/product/8063794,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=11,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=8,http://frys.com/product/8063794#detailed,http://www.walmart.com/ip/Sony-Lbt-gpx555-Mini-Hi-fi-System-1800-W-Rms-Black-Cd-Player-30-Channel-s-Am-Fm-Video-Cd-Sacd-Dvd-Video-2.1-Speaker-s-Cd-da-Mp3/46145643,http://www.newegg.com/Product/Product.aspx?Item=9SIA0AJ3MD4812,http://www.newegg.com/Product/Product.aspx?Item=N82E16882105737,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=5,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037#tab=buyingOptions?bof=openbox,http://www.bestbuy.com:80/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037#tabbed-customerreviews,http://www.bestbuy.com:80/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037#tabbed-customerreviews,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=3,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037&reviewsPage=2,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=6,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037&reviewsPage=4,http://www.bestbuy.com/site/sony-1800w-wireless-bookshelf-stereo-system-black/8038037.p?id=1219307881697&skuId=8038037&reviewsPage=3,http://www.walmart.com/ip/Sony-LBT-GPX555-Mini-System-with-Bluetooth-and-NFC/40588069?action=product_interest&action_type=title&item_id=40588069&placement_id=irs-2-b1&strategy=TIC&visitor_id&category=&customer_id_enc&config_id=2&parent_item_id=20370545&parent_anchor_item_id=20370545&bucket_id=irsbucketdefault&beacon_version=1.0.1&findingMethod=p13n,http://www.walmart.com/ip/Sony-LBT-GPX555-Mini-System-with-Bluetooth-and-NFC/40588069?action=product_interest&action_type=title&item_id=40588069&placement_id=irs-2-b1&strategy=TIC&visitor_id&category=&customer_id_enc&config_id=2&parent_item_id=20370545&parent_anchor_item_id=20370545&bucket_id=irsbucketdefault&beacon_version=1.0.1&findingMethod=p13n#reviews,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=7,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=12,http://reviews.bestbuy.com/3545/8038037/reviews.htm?format=embedded&page=13", "missing": "0", "distinct": "19"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "sourceURLs"}, {"transformationsData": [{"feature_label": "upc", "transformation_label": "String Indexer"}], "feature": "upc", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "890000000000.0", "stddev": "0.0", "min": " it works most of the time and is a quicker way to \"\"sign-on\"\" than entering my 20 character password!The keyboard easily comes off and on with the tablet and it is magnetized allowing for the keyboard to be at an angle compatible with typing while I was in-flight.\"", "max": "wNg11", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "upc"}, {"transformationsData": [{"feature_label": "weight", "transformation_label": "String Indexer"}], "feature": "weight", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "27242902695.0", "stddev": "0.0", "min": " it can make some songs sound a little \"\"muddy\"\" with an overemphasis on lows.+ NFC pairing: If your phone features an NFC chip", "max": "http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=13,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=14,https://www.bestbuy.com/site/sony-xb30-portable-bluetooth-speaker-black/5744203.p?skuId=5744203,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=15,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=16,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=17,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=18,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=19,http://www.bestbuy.com/site/products/5744203.p,http://www.ebay.com/itm/Sony-SRS-XB30-Portable-Bluetooth-Wireless-Speaker-Black/272855583204,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=20,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=21,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=22,https://www.amazon.com/Sony-Portable-Wireless-Speaker-Bluetooth/dp/B01MUG578I/,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=23,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=46,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=47,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=48,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=49,https://www.bestbuy.com/site/products/5744203.p,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=50,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=51,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=52,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=53,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=54,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=10,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=55,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=11,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=56,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=12,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=35,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=36,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=37,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=38,https://www.amazon.com/Sony-SRSXB30-BLK-Portable-Bluetooth/dp/B01MUG578I/,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=39,https://www.amazon.com/Sony-Bluetooth-SRS-XB30-BLK-Refurbished/dp/B072MP2CFG/,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=4,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=5,https://www.bhphotovideo.com/c/product/1319172-REG/sony_srsxb30_blk_srsxb30_extra_bass_wireless.html,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=2,https://www.walmart.com/ip/SRSXB30-BLK/697156213,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=3,http://www.bestbuy.com/site/sony-xb30-portable-bluetooth-speaker-black/5744203.p?skuId=5744203,http://www.ebay.com/itm/Sony-XB30-Portable-Wireless-Speaker-with-Bluetooth-Black-2017-model/311881660787?hash=item489d987973%3Ag%3AstQAAOSwR29ZKBj4,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=40,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=41,https://www.walmart.com/ip/SONY-SRS-XB30-BLK-Portable-Wireless-Speaker/697156213,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=42,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=8,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=43,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=9,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=44,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=6,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=45,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=7,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=24,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=25,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=26,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=27,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=28,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=29,http://www.ebay.com/itm/Sony-SRS-XB30-Portable-Bluetooth-Wireless-Speaker-Black-SRSXB30/282573811327,http://www.ebay.com/itm/Sony-SRS-XB30-Portable-Bluetooth-Wireless-Speaker-Black-SRSXB30/282573811327?hash=item41cab62e7f%3Ag%3AYy4AAOSwRvdZa5oE,http://www.walmart.com/ip/SONY-SRS-XB30-BLK-Portable-Wireless-Speaker/697156213,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=30,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=31,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=32,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=33,https://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded,http://reviews.bestbuy.com/3545/5744203/reviews.htm?format=embedded&page=34", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "weight"}]}))

	#transformationPostExecutionHook(regressionmodllautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run regressionmodllHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(regressionmodllautofe, ["reviewsdoRecommend", "reviewsnumHelpful", "reviewsrating", "id_stringindexer", "asins_stringindexer", "brand_stringindexer", "categories_stringindexer", "colors_stringindexer", "dateAdded_dayofmonth", "dateAdded_month", "dateAdded_year", "dateUpdated_dayofmonth", "dateUpdated_month", "dateUpdated_year", "dimension_stringindexer", "ean_binarizer", "imageURLs_stringindexer", "keys_stringindexer", "manufacturer_stringindexer", "manufacturerNumber_stringindexer", "name_stringindexer", "primaryCategories_stringindexer", "reviewsdate_dayofmonth", "reviewsdate_month", "reviewsdate_year", "reviewsdateSeen_stringindexer", "reviewssourceURLs_stringindexer", "reviewstitle_stringindexer", "reviewsusername_stringindexer", "sourceURLs_stringindexer", "upc_stringindexer", "weight_stringindexer"], "reviewstext_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

